#  Sprint8 アンサンブル学習

In [427]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns
# from numpy import linalg as LA
import copy
sns.set()
%matplotlib inline
import time
import math

from sklearn.model_selection import train_test_split
import random
from sklearn.decomposition import PCA
from sklearn.cluster import KMeans
from sklearn.preprocessing import StandardScaler
from sklearn.model_selection import cross_val_score
from sklearn.metrics import mean_squared_error, classification_report
from sklearn.neural_network import MLPClassifier
from sklearn.svm import SVR
from sklearn.svm import SVC
from lightgbm import LGBMClassifier
from sklearn.linear_model import LinearRegression
from sklearn.model_selection import cross_validate
from sklearn.ensemble import RandomForestClassifier
from sklearn.datasets import make_classification
from sklearn.model_selection import KFold

In [466]:
from sklearn.metrics import accuracy_score
from sklearn.metrics import precision_score
from sklearn.metrics import recall_score
from sklearn.metrics import f1_score
from sklearn.metrics import confusion_matrix

def scores(y_test,y_pred):
    print("accuracy",accuracy_score(y_test,y_pred))
    print("precision",precision_score(y_test,y_pred))
    print("recall",recall_score(y_test,y_pred))
    print("f1_score",f1_score(y_test,y_pred))
    print("conf_matrix",confusion_matrix(y_test,y_pred))
    
def crossval_score(model, X_df, y_df, n_splits): 
    score = cross_val_score(model, X_df, y_df,cv=n_splits)
    print('Cross-Validation scores: {}'.format(score))
    print('Average score: {}'.format(np.mean(score)))

In [665]:
# 小さなデータセットの用意
train_data = pd.read_csv('train.csv')
X_df = train_data[['GrLivArea','YearBuilt']]
y_df = train_data['SalePrice']

X_data = copy.deepcopy(X_df)
scaler = StandardScaler()
scaler.fit(X_data)
X_data_std = pd.DataFrame(scaler.transform(X_data), columns=X_data.columns)
X_data_log = np.log(X_data)
display(X_df.shape[0])
print(X_data.isnull().all())

/Users/akishimasaki/.pyenv/versions/anaconda3-2019.03/lib/python3.7/site-packages/sklearn/preprocessing/data.py:645: DataConversionWarning: Data with input dtype int64 were all converted to float64 by StandardScaler.
  return self.partial_fit(X, y)
/Users/akishimasaki/.pyenv/versions/anaconda3-2019.03/lib/python3.7/site-packages/ipykernel_launcher.py:9: DataConversionWarning: Data with input dtype int64 were all converted to float64 by StandardScaler.
  if __name__ == '__main__':


1460

GrLivArea    False
YearBuilt    False
dtype: bool


In [386]:
X_train, X_test, y_train, y_test = train_test_split(X_df,y_df, test_size=0.20)

In [383]:
def MSE(y_test,y_pred):
    mse = mean_squared_error(y_test, y_pred, multioutput='uniform_average')
    return print("MSE:",mse)

In [ ]:
'''
from sklearn.model_selection import GridSearchCV

# パラメータを dict 型で指定
param_grid = {'C': [0.001, 0.01, 0.1, 1, 10, 100],  'gamma' : [0.001, 0.01, 0.1, 1, 10, 100]}

# validation set は GridSearchCV が自動で作成してくれるため，
# training set と test set の分割のみを実行すればよい
X_train, X_test, y_train, y_test = train_test_split(X_df, y_df)

grid_search = GridSearchCV(SVC(), param_grid, cv=5)

# fit 関数を呼ぶことで交差検証とグリッドサーチがどちらも実行される
grid_search.fit(X_train, y_train)

print('Test set score: {}'.format(grid_search.score(X_test, y_test)))
print('Best parameters: {}'.format(grid_search.best_params_))
print('Best cross-validation: {}'.format(grid_search.best_score_))
'''

単一モデル

In [668]:
#線形回帰
lr = LinearRegression()
lr.fit(X_train,y_train)
y_pred_LR = lr.predict(X_test)
MSE(y_test,y_pred_LR)

MSE: 1931052698.4079645


In [389]:
#SVR
svr = SVR(gamma='scale', C=1.0, epsilon=0.2)
svr.fit(X_train, y_train)
y_pred_SVR = svm.predict(X_test)
MSE(y_test,y_pred_SVR)

MSE: 10605743997.417809


In [390]:
#randomforest
rforest = RandomForestClassifier(n_estimators=5, 
                                 max_depth=2,random_state=0)
rforest.fit(X_train, y_train)
y_pred_Forest = rforest.predict(X_test)
MSE(y_test,y_pred_Forest)

MSE: 3367437741.2089043


In [391]:
#LightGB
lgb = LGBMClassifier()
lgb.fit(X_train, y_train)
y_pred_LGB = lgb.predict(X_test)
MSE(y_test,y_pred_LGB)

MSE: 2491989405.2054796


In [392]:
#ニューラルネットワーク
clf = MLPClassifier(hidden_layer_sizes=(100, ), alpha=0.0001) 
clf.fit(X_train, y_train)
y_pred_MLPC = clf.predict(X_test)
MSE(y_test,y_pred_MLPC)

MSE: 5583978487.784246


## 【問題1】ブレンディングのスクラッチ実装
>単一モデルより精度があがる例を **最低3つ** 示してください。精度があがるとは、検証用データに対する平均二乗誤差（MSE）が小さくなることを指します。

In [393]:
'''
N個の多様なモデル
(手法（例：線形回帰、SVM、決定木、ニューラルネットワークなど）,
ハイパーパラメータ（例：SVMのカーネルの種類、重みの初期値など）,
入力データの前処理の仕方（例：標準化、対数変換、PCAなど）)を独立して学習

推定結果を重み付けした上で足し合わせる
'''
#重要なのはそれぞれのモデルが大きく異なること

'\nN個の多様なモデル\n(手法（例：線形回帰、SVM、決定木、ニューラルネットワークなど）,\nハイパーパラメータ（例：SVMのカーネルの種類、重みの初期値など）,\n入力データの前処理の仕方（例：標準化、対数変換、PCAなど）)を独立して学習\n\n推定結果を重み付けした上で足し合わせる\n'

In [178]:
# SVM x ニューラルネットワーク x 対数変換あるなし

#SVR
svr = SVR(gamma='scale', C=1.0, epsilon=0.2)
svr.fit(X_train, y_train)
y_pred_SVR = svm.predict(X_test)
MSE(y_test,y_pred_SVR)
# crossval_score(clf, X_df, y_df)

#ニューラルネットワーク
clf = MLPClassifier(hidden_layer_sizes=(100, ), alpha=0.0001) 
clf.fit(X_train, y_train)
y_pred_MLPC = clf.predict(X_test)
MSE(y_test,y_pred_MLPC)
# crossval_score(clf, X_df, y_df)

#対数変換あり　X_data_log
X_train_log, X_test_log, y_train, y_test = train_test_split(X_data_log,y_df, test_size=0.20)

svr.fit(X_train_log, y_train)
y_pred_SVR_log = svm.predict(X_test_log)
MSE(y_test,y_pred_SVR_log)
# crossval_score(clf, X_data_log, y_df)

# clf.fit(X_train_log, y_train)
# y_pred_MLPC_log = clf.predict(X_test_log)
# MSE(y_test,y_pred_MLPC_log)
# crossval_score(clf, X_data_log, y_df)

'''
独立して学習、推定した結果を重み付けした上で足し合わせる。
最も単純には平均をとります。
個々のMSEよりも下がるMSEになる
'''
# weights = [0.01,0.1,0.05,0.1]
# blend_trying = weights[0] * y_pred_SVM + weights[1] * y_pred_MLPC + weights[2] * y_pred_SVM_log + weights[3] * y_pred_MLPC_log
blend_mean_1 = (y_pred_SVR + y_pred_MLPC + y_pred_SVR_log + y_pred_MLPC_log)/3
# print(blend_mean)
print("Blendy1")
MSE(y_test,blend_mean_1)

MSE: 9266730841.736301
MSE: 8473890630.092465
MSE: 8360929059.147261
Blendy1
MSE: 6685294761.894596


In [180]:
#標準化なし x ニューラルネットワーク　 x LightGB  x SVR
clf = MLPClassifier(hidden_layer_sizes=(100, ), alpha=0.0001) 
clf.fit(X_train, y_train)
y_pred_MLPC = clf.predict(X_test)
MSE(y_test,y_pred_MLPC)

lgb = LGBMClassifier()
lgb.fit(X_train, y_train)
y_pred_LGB = lgb.predict(X_test)
MSE(y_test,y_pred_LGB)

svr.fit(X_train, y_train)
y_pred_SVM = svr.predict(X_test)
MSE(y_test,y_pred_SVR)

# clf = MLPClassifier(hidden_layer_sizes=(100, ), alpha=0.0001) 
# clf.fit(X_train_std, y_train)
# y_pred_MLPC_std = clf.predict(X_test_std)
# MSE(y_test,y_pred_MLPC_std)

# lgb = LGBMClassifier()
# lgb.fit(X_train_std, y_train)
# y_pred_LGB_std = lgb.predict(X_test_std)
# MSE(y_test,y_pred_LGB_std)

blend_mean_2 = (y_pred_MLPC + y_pred_LGB + y_pred_SVR)/3
# print(blend_mean)
print("Blendy2")
MSE(y_test,blend_mean_2)

MSE: 13438564931.335617
MSE: 11799545002.109589
MSE: 8559343048.873287
Blendy2
MSE: 7963911370.503425


In [196]:
# 標準化して、線形回帰 x randomforest x LGBM

X_train_std, X_test_std, y_train, y_test = train_test_split(X_data_std,y_df, test_size=0.20)

lr.fit(X_train_std, y_train)
y_pred_LR_std = lr.predict(X_test_std)
MSE(y_test,y_pred_LR_std)

rforest = RandomForestClassifier(n_estimators=5, 
                                 max_depth=2,random_state=0)
rforest.fit(X_train_std, y_train)
y_pred_Forest_std = rforest.predict(X_test_std)
MSE(y_test,y_pred_Forest_std)

lgb.fit(X_train_std, y_train)
y_pred_LGB_std = lgb.predict(X_test_std)
MSE(y_test,y_pred_LGB_std)

blend_mean_3 = 0.8 * y_pred_LR_std + 0.1 * y_pred_LGB_std + 0.1 * y_pred_Forest_std #重みは合計１(100%)になれば良い
# print(blend_mean)
print("Blendy3")
MSE(y_test,blend_mean_3)

MSE: 1753480330.8051112
MSE: 2589001206.691781
MSE: 2704652506.390411
Blendy3
MSE: 1679242015.9683964


In [ ]:
#結論: 標準化ぱない。線形回帰速い。Lgb遅い。
#Blending中にいい組み合わせを発見できた。

## 【問題2】バギングのスクラッチ実装
>学習データから重複を許した上でランダムに抜き出すことで、N種類のサブセット（ ブートストラップサンプル ）を作り出します。それらによってモデルをN個学習し、推定結果の平均をとります。ブレンディングと異なり、それぞれの重み付けを変えることはありません。

In [604]:
#バギングとRandomForest

#data をランダムに8割抜き出す。　x N回　　　　　　
'''インデックスだけ先にランダムで作っちゃう
train_test_split shuffle = Trueがあるが、
これは非復元抽出になるためブートストラップサンプルではない、、、
(一回ごとにカードを切り直す訳ではないため) #replace=True
'''
#N個のpred の平均をとる。

def bagging_MSE(model,X_df,y_df,n=3):
    X_train, X_test, y_train, y_test = train_test_split(X_df,y_df, test_size=0.20)
    y_pred_array = np.zeros_like(y_test)
    for i in range(n):
        #0から X_df.shape[0]までの中からランダムで整数値を出力
        index_random = [random.randint(0,X_train.shape[0]-1)
                    for i in range( X_train.shape[0])]
        X_train= X_train.iloc[index_random,:]
        y_train= y_train.iloc[index_random]

        model.fit(X_train,y_train)
        y_pred = model.predict(X_test)
        y_pred_array = np.concatenate([y_pred_array ,y_pred])    
    
    y_pred_arrays = np.delete(y_pred_array.reshape((n+1,len(y_test))),0,0).T
    y_pred_bagging = np.mean(y_pred_arrays,axis=1)
    mse = MSE(y_test,y_pred_bagging)
    return mse

In [605]:
print("SVR")
bagging_MSE(svr,X_df,y_df,n=3)
print("Nnetwork")
bagging_MSE(clf,X_df,y_df,n=3)
'''
線形回帰   MSE: 1931052698.4079645
SVR          MSE: 10605743997.417809
Rforest     MSE: 3367437741.2089043
LightGB    MSE: 2491989405.2054796
Nnetwork MSE: 5583978487.784246
'''

SVR
MSE: 7014354894.102712
Nnetwork
MSE: 3989458806.16971


'\n線形回帰   MSE: 1931052698.4079645\nSVR          MSE: 10605743997.417809\nRforest     MSE: 3367437741.2089043\nLightGB    MSE: 2491989405.2054796\nNnetwork MSE: 5583978487.784246\n'

## 【問題3】スタッキングのスクラッチ実装

In [ ]:
'''
blend data を　(何層か回せるようにする) →　最後になんでもいいのzにfit　,MSE

推論した方のblend data を　(何層か回せるようにする) →　zでpred , MSE
'''

In [666]:
#train_test やってからkfoldでvalidaton
model = [svr,clf,rforest]
X_array = np.array(X_df)
y_array = np.array(y_df)


def stacking(model,X_array,y_array,n_splits=4):
    kfold = KFold(n_splits=n_splits) #インスタンス化
    X_train, X_test, y_train, y_test = train_test_split(X_array,y_array, test_size=0.20)
    blend_data_array = np.zeros_like(y_train) #blend data格納
    blend_preds = np.zeros_like(y_test) #y_pred格納
    for i in range(len(model)):
        blend_pred_array = np.zeros_like(y_test) #blend preds格納
        print("model:", i)
        #kfoldでvalidatonをn個作る
        for train, test in kfold.split(X_train):
            train_index = train
            test_index = test
            X_train_train = X_train[train_index] #train data
            X_train_val = X_train[test_index] # validation data
            y_train_train = y_train[train_index] #学習用train data
            y_train_val = y_train[test_index]  # validation data 正解ラベル

            model[i].fit(X_train_train,y_train_train) #fit
            #fit　して　crossval_score
            
            crossval_score(model[i], X_train, y_train, n_splits) 
            X_val_pred = model[i].predict(X_train_val) #blend data素材の原材料   
            '''
            ここの学習済みmodel 用いて kfoldのfor 内でpred fase　も
            
            '''
            y_pred = model[i].predict(X_test) #blend pred素材の1つ
            blend_data_array= np.concatenate([blend_data_array ,X_val_pred])    
            np.append(blend_data_array, X_val_pred,axis=0)  #blend data
            
            blend_pred_array= np.concatenate([blend_pred_array ,y_pred])    
            np.append(blend_pred_array, y_pred, axis=0)  #blend pred素材
            
        blend_preds_array = np.delete(blend_pred_array.reshape(n_splits+1,len(X_test)),0,0).T
        blend_pred = np.mean(blend_preds_array,axis=1) 
        blend_preds = np.concatenate([blend_preds ,blend_pred])   
        np.append(blend_preds, blend_pred,axis=0)  #blend pred
    
    blend_data_array = np.delete(blend_data_array
                                 .reshape(len(model)+1,len(X_train)),0,0).T
    blend_preds = np.delete(blend_preds
                                 .reshape(len(model)+1,len(y_test)),0,0).T
    
    lr.fit(blend_data_array,y_train) #線形回帰でMSE算出
    final_pred = lr.predict(blend_preds)
    MSE(y_test,final_pred)
    
    return blend_data_array, blend_preds

In [658]:
model = [svr,clf,rforest]
blend_data_array, blend_preds = stacking(model,X_array,y_array,n_splits=4)

model: 0
Cross-Validation scores: [-0.01832231 -0.05876026 -0.08130152 -0.03917829]
Average score: -0.04939059542462526
Cross-Validation scores: [-0.01832231 -0.05876026 -0.08130152 -0.03917829]
Average score: -0.04939059542462526
Cross-Validation scores: [-0.01832231 -0.05876026 -0.08130152 -0.03917829]
Average score: -0.04939059542462526
Cross-Validation scores: [-0.01832231 -0.05876026 -0.08130152 -0.03917829]
Average score: -0.04939059542462526
model: 1


/Users/akishimasaki/.pyenv/versions/anaconda3-2019.03/lib/python3.7/site-packages/sklearn/model_selection/_split.py:652: Warning: The least populated class in y has only 1 members, which is too few. The minimum number of members in any class cannot be less than n_splits=4.
  % (min_groups, self.n_splits)), Warning)


Cross-Validation scores: [0.00618238 0.01123596 0.00628931 0.03157895]
Average score: 0.013821647704271191


/Users/akishimasaki/.pyenv/versions/anaconda3-2019.03/lib/python3.7/site-packages/sklearn/model_selection/_split.py:652: Warning: The least populated class in y has only 1 members, which is too few. The minimum number of members in any class cannot be less than n_splits=4.
  % (min_groups, self.n_splits)), Warning)


Cross-Validation scores: [0.00463679 0.         0.01886792 0.01052632]
Average score: 0.008507756370015763


...

Cross-Validation scores: [0.0015456  0.01123596 0.01257862 0.01052632]
Average score: 0.008971620562987635
model: 2
Cross-Validation scores: [0.00927357 0.01123596 0.02515723 0.03157895]
Average score: 0.019311426363394578


/Users/akishimasaki/.pyenv/versions/anaconda3-2019.03/lib/python3.7/site-packages/sklearn/model_selection/_split.py:652: Warning: The least populated class in y has only 1 members, which is too few. The minimum number of members in any class cannot be less than n_splits=4.
  % (min_groups, self.n_splits)), Warning)
/Users/akishimasaki/.pyenv/versions/anaconda3-2019.03/lib/python3.7/site-packages/sklearn/model_selection/_split.py:652: Warning: The least populated class in y has only 1 members, which is too few. The minimum number of members in any class cannot be less than n_splits=4.
  % (min_groups, self.n_splits)), Warning)


Cross-Validation scores: [0.00927357 0.01123596 0.02515723 0.03157895]
Average score: 0.019311426363394578
Cross-Validation scores: [0.00927357 0.01123596 0.02515723 0.03157895]
Average score: 0.019311426363394578


/Users/akishimasaki/.pyenv/versions/anaconda3-2019.03/lib/python3.7/site-packages/sklearn/model_selection/_split.py:652: Warning: The least populated class in y has only 1 members, which is too few. The minimum number of members in any class cannot be less than n_splits=4.
  % (min_groups, self.n_splits)), Warning)
/Users/akishimasaki/.pyenv/versions/anaconda3-2019.03/lib/python3.7/site-packages/sklearn/model_selection/_split.py:652: Warning: The least populated class in y has only 1 members, which is too few. The minimum number of members in any class cannot be less than n_splits=4.
  % (min_groups, self.n_splits)), Warning)


Cross-Validation scores: [0.00927357 0.01123596 0.02515723 0.03157895]
Average score: 0.019311426363394578
MSE: 2955844525.2748737


In [ ]:
'''
model = [SVR, Nnetwork, Rforest]を用いたところ、

線形回帰   MSE: 1931052698.4079645
SVR          MSE: 10605743997.417809
Rforest     MSE: 3367437741.2089043
LightGB    MSE: 2491989405.2054796
Nnetwork MSE: 5583978487.784246

より、元のモデルより良いStackingモデルが得られた。
'''

In [456]:
#クロスバリデーション（交差検証） をする。 
#分割を複数回行い、それぞれに対して学習と検証を行う方法です。
#複数回の分割のためにscikit-learnにはKFoldクラスが用意されています。

In [667]:
X_array_std = np.array(X_data_std)
model = [lr,rforest]
stacking(model,X_array_std,y_array,n_splits=4)  #MSE: 2399808646.8215013

model: 0
Cross-Validation scores: [0.68196062 0.72629909 0.65892911 0.60699277]
Average score: 0.668545397923322
Cross-Validation scores: [0.68196062 0.72629909 0.65892911 0.60699277]
Average score: 0.668545397923322
Cross-Validation scores: [0.68196062 0.72629909 0.65892911 0.60699277]
Average score: 0.668545397923322
Cross-Validation scores: [0.68196062 0.72629909 0.65892911 0.60699277]
Average score: 0.668545397923322
model: 1
Cross-Validation scores: [0.01095462 0.01960784 0.04705882 0.02884615]
Average score: 0.026616859275309983


/Users/akishimasaki/.pyenv/versions/anaconda3-2019.03/lib/python3.7/site-packages/sklearn/model_selection/_split.py:652: Warning: The least populated class in y has only 1 members, which is too few. The minimum number of members in any class cannot be less than n_splits=4.
  % (min_groups, self.n_splits)), Warning)
/Users/akishimasaki/.pyenv/versions/anaconda3-2019.03/lib/python3.7/site-packages/sklearn/model_selection/_split.py:652: Warning: The least populated class in y has only 1 members, which is too few. The minimum number of members in any class cannot be less than n_splits=4.
  % (min_groups, self.n_splits)), Warning)


Cross-Validation scores: [0.01095462 0.01960784 0.04705882 0.02884615]
Average score: 0.026616859275309983
Cross-Validation scores: [0.01095462 0.01960784 0.04705882 0.02884615]
Average score: 0.026616859275309983


/Users/akishimasaki/.pyenv/versions/anaconda3-2019.03/lib/python3.7/site-packages/sklearn/model_selection/_split.py:652: Warning: The least populated class in y has only 1 members, which is too few. The minimum number of members in any class cannot be less than n_splits=4.
  % (min_groups, self.n_splits)), Warning)
/Users/akishimasaki/.pyenv/versions/anaconda3-2019.03/lib/python3.7/site-packages/sklearn/model_selection/_split.py:652: Warning: The least populated class in y has only 1 members, which is too few. The minimum number of members in any class cannot be less than n_splits=4.
  % (min_groups, self.n_splits)), Warning)


Cross-Validation scores: [0.01095462 0.01960784 0.04705882 0.02884615]
Average score: 0.026616859275309983
MSE: 2399808646.8215013


(array([[183409.84675361, 140000.        ],
        [132688.10556718, 140000.        ],
        [133755.26250238, 140000.        ],
        ...,
        [210925.82553038, 140000.        ],
        [119878.11316358, 140000.        ],
        [209211.01148884, 140000.        ]]),
 array([[246919.35949527, 187500.        ],
        [320621.67915088, 162125.        ],
        [285378.77490274, 191250.        ],
        [151712.25380489, 137250.        ],
        [197661.84544144, 161250.        ],
        [131720.89052918, 138750.        ],
        [143342.93322902, 138750.        ],
        [222985.53290964, 191250.        ],
        [202105.661863  , 187500.        ],
        [ 85094.98147693, 113750.        ],
        [235681.68867552, 187500.        ],
        [304965.76188317, 203750.        ],
        [227226.78488048, 191250.        ],
        [ 66318.51477973, 111000.        ],
        [126449.33218841, 132500.        ],
        [153991.63754492, 151250.        ],
        [134410.4

In [660]:
model = [lr,lgb,rforest]
stacking(model,X_array,y_array,n_splits=4) #MSE: 2328149499.0840955

model: 0
Cross-Validation scores: [0.67022238 0.56454093 0.67911422 0.62521836]
Average score: 0.6347739717920808
Cross-Validation scores: [0.67022238 0.56454093 0.67911422 0.62521836]
Average score: 0.6347739717920808
Cross-Validation scores: [0.67022238 0.56454093 0.67911422 0.62521836]
Average score: 0.6347739717920808
Cross-Validation scores: [0.67022238 0.56454093 0.67911422 0.62521836]
Average score: 0.6347739717920808
model: 1


/Users/akishimasaki/.pyenv/versions/anaconda3-2019.03/lib/python3.7/site-packages/sklearn/model_selection/_split.py:652: Warning: The least populated class in y has only 1 members, which is too few. The minimum number of members in any class cannot be less than n_splits=4.
  % (min_groups, self.n_splits)), Warning)


Cross-Validation scores: [0.0078125  0.02681992 0.0060241  0.02970297]
Average score: 0.017589872513554844


/Users/akishimasaki/.pyenv/versions/anaconda3-2019.03/lib/python3.7/site-packages/sklearn/model_selection/_split.py:652: Warning: The least populated class in y has only 1 members, which is too few. The minimum number of members in any class cannot be less than n_splits=4.
  % (min_groups, self.n_splits)), Warning)


Cross-Validation scores: [0.0078125  0.02681992 0.0060241  0.02970297]
Average score: 0.017589872513554844


...

Cross-Validation scores: [0.0078125  0.02681992 0.0060241  0.02970297]
Average score: 0.017589872513554844
model: 2
Cross-Validation scores: [0.00625    0.01532567 0.01204819 0.01980198]
Average score: 0.013356460866797107


/Users/akishimasaki/.pyenv/versions/anaconda3-2019.03/lib/python3.7/site-packages/sklearn/model_selection/_split.py:652: Warning: The least populated class in y has only 1 members, which is too few. The minimum number of members in any class cannot be less than n_splits=4.
  % (min_groups, self.n_splits)), Warning)
/Users/akishimasaki/.pyenv/versions/anaconda3-2019.03/lib/python3.7/site-packages/sklearn/model_selection/_split.py:652: Warning: The least populated class in y has only 1 members, which is too few. The minimum number of members in any class cannot be less than n_splits=4.
  % (min_groups, self.n_splits)), Warning)


Cross-Validation scores: [0.00625    0.01532567 0.01204819 0.01980198]
Average score: 0.013356460866797107
Cross-Validation scores: [0.00625    0.01532567 0.01204819 0.01980198]
Average score: 0.013356460866797107
Cross-Validation scores: [0.00625    0.01532567 0.01204819 0.01980198]
Average score: 0.013356460866797107
MSE: 2328149499.0840955


/Users/akishimasaki/.pyenv/versions/anaconda3-2019.03/lib/python3.7/site-packages/sklearn/model_selection/_split.py:652: Warning: The least populated class in y has only 1 members, which is too few. The minimum number of members in any class cannot be less than n_splits=4.
  % (min_groups, self.n_splits)), Warning)
/Users/akishimasaki/.pyenv/versions/anaconda3-2019.03/lib/python3.7/site-packages/sklearn/model_selection/_split.py:652: Warning: The least populated class in y has only 1 members, which is too few. The minimum number of members in any class cannot be less than n_splits=4.
  % (min_groups, self.n_splits)), Warning)


(array([[193229.84874774, 147400.        , 190000.        ],
        [232166.11924365, 244000.        , 190000.        ],
        [137813.17083791, 177000.        , 145000.        ],
        ...,
        [ 86939.83558896,  94000.        , 140000.        ],
        [204327.12217526, 187100.        , 173000.        ],
        [129917.51675199, 149900.        , 135000.        ]]),
 array([[211488.41635918, 184750.        , 185750.        ],
        [214515.07825774, 273341.5       , 177500.        ],
        [ 92294.52257579, 119000.        , 157500.        ],
        [224500.01551579, 226250.        , 185000.        ],
        [244567.27308518, 269250.        , 185750.        ],
        [142188.24743206, 146375.        , 156000.        ],
        [128908.9583201 , 120375.        , 157500.        ],
        [169217.14491686, 177500.        , 177500.        ],
        [207234.14596092, 195850.        , 185750.        ],
        [153047.96869031, 162750.        , 157500.        ],
        [